In [1]:
from federation.generate_scenarios import query_time_series_data, fill_null_remove_outliers, generate_scenarios
from schema.schema import Irradiation, WindSpeed, Temperature
DB_CACHE_FILE = r"../.cache/interim/time_series_schema.db"
data = query_time_series_data(DB_CACHE_FILE, alt=2510, tables=[Irradiation])["Irradiation"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios1 = generate_scenarios(data)
data = query_time_series_data(DB_CACHE_FILE, alt=2510, tables=[WindSpeed])["WindSpeed"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios2 = generate_scenarios(data)
data = query_time_series_data(DB_CACHE_FILE, alt=2510, tables=[Temperature])["Temperature"]
data = fill_null_remove_outliers(data, d_time="1h", z_score=10)
data_scenarios3 = generate_scenarios(data)

In [2]:
import polars as pl
data_df1 = data_scenarios1.filter(pl.col("scenario")=="min").select(["timestamp", "value"]).to_pandas().set_index("timestamp")["value"].asfreq("1H").fillna(0)
data_df2 = data_scenarios2.filter(pl.col("scenario")=="min").select(["timestamp", "value"]).to_pandas().set_index("timestamp")["value"].asfreq("1H").fillna(0)
data_df3 = data_scenarios3.filter(pl.col("scenario")=="min").select(["timestamp", "value"]).to_pandas().set_index("timestamp")["value"].asfreq("1H").fillna(0)

In [3]:
import warnings
import numpy as np
from sktime.forecasting.base import ForecastingHorizon
from pycaret.datasets import get_data
from pycaret.time_series import *
exp1 = TSForecastingExperiment()
exp2 = TSForecastingExperiment()
exp3 = TSForecastingExperiment()
warnings.filterwarnings("ignore")
fh = ForecastingHorizon(values=np.arange(24,48), is_relative=True)
s1 = exp1.setup(
    data=data_df1, fh=fh, fold=12, session_id=41
)
s2 = exp2.setup(
    data=data_df2, fh=fh, fold=12, session_id=42
)
s3 = exp3.setup(
    data=data_df3, fh=fh, fold=12, session_id=43
)

,Description,Value
0,session_id,41
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8736, 1)"
5,Transformed data shape,"(8736, 1)"
6,Transformed train set shape,"(8689, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8736, 1)"
5,Transformed data shape,"(8736, 1)"
6,Transformed train set shape,"(8689, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Description,Value
0,session_id,43
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(8736, 1)"
5,Transformed data shape,"(8736, 1)"
6,Transformed train set shape,"(8689, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [4]:
from sktime.forecasting.auto_reg import AutoREG  
autoreg_sktime = AutoREG(lags=[i for i in range(24, 72)], trend="n")
arima1 = exp1.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})
arima2 = exp2.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})
arima3 = exp3.create_model(autoreg_sktime, fit_kwargs={"fh": "{fh}"})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:34:37
Status,. . . . . . . . . . . . . . . . . .,Fitting 12 Folds
Estimator,. . . . . . . . . . . . . . . . . .,AutoREG


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2014-12-16 01:00,0.1525,0.1080,8.8621,13.7913,1.5734,0.9890,0.9552
1,2014-12-17 01:00,0.1260,0.0885,7.3038,11.2850,2.6881,0.8955,0.9704
2,2014-12-18 01:00,0.3573,0.3031,20.6650,38.6097,2.5455,0.9599,0.8262
3,2014-12-19 01:00,0.4242,0.3609,24.4741,45.9056,2.4436,1.1005,0.7811
4,2014-12-20 01:00,0.5067,0.4570,29.1747,58.0598,2.0026,1.0228,0.7628
5,2014-12-21 01:00,0.3422,0.3132,19.6487,39.7296,2.4273,0.8567,0.8791
6,2014-12-22 01:00,0.2762,0.2301,15.8299,29.1542,2.5014,0.9163,0.9399
7,2014-12-23 01:00,0.2693,0.1971,15.3965,24.9422,2.1733,0.7629,0.9100
8,2014-12-24 01:00,0.1727,0.1182,9.8460,14.9302,3.4360,0.9981,0.9821
9,2014-12-25 01:00,0.3057,0.2508,17.3990,31.6445,2.5385,0.9148,0.9247


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2011-12-16 01:00,2.3062,1.8464,2.6329,2.9169,0.4746,0.4708,-0.3335
1,2011-12-17 01:00,2.4846,1.8570,2.8480,2.9454,1.0044,0.6304,-10.3518
2,2011-12-18 01:00,2.5495,1.9590,2.9329,3.1232,0.9364,0.6077,-58.2722
3,2011-12-19 01:00,0.8307,0.7913,0.9609,1.2720,0.1733,0.1954,-1.5116
4,2011-12-20 01:00,0.7801,0.6625,0.9012,1.0638,0.2112,0.2347,-0.7620
5,2011-12-21 01:00,0.4603,0.4101,0.5326,0.6590,0.1270,0.1310,-0.1926
6,2011-12-22 01:00,0.2859,0.2654,0.3310,0.4265,0.0937,0.0876,-0.0158
7,2011-12-23 01:00,3.2554,2.6335,3.7667,4.2285,0.4470,0.6051,-4.0755
8,2011-12-24 01:00,0.5447,0.4923,0.6292,0.7894,0.2133,0.1842,-0.1898
9,2011-12-25 01:00,2.0293,1.5467,2.3600,2.5031,0.8088,0.5531,-28.8123


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2010-12-16 01:00,0.7641,0.6866,1.6432,1.9812,0.1172,0.1089,-1.0888
1,2010-12-17 01:00,1.1822,1.0314,2.5400,2.9736,0.1449,0.1608,-7.9822
2,2010-12-18 01:00,1.4612,1.3009,3.1429,3.7524,0.3129,0.2582,-2.1551
3,2010-12-19 01:00,4.0247,3.0531,8.6664,8.8105,1.4221,0.8053,-40.9421
4,2010-12-20 01:00,1.7049,1.3248,3.6914,3.8455,0.6113,0.4542,-3.9292
5,2010-12-21 01:00,2.0327,1.5817,4.4139,4.6035,1.6565,0.8598,-27.5060
6,2010-12-22 01:00,1.6821,1.4899,3.6506,4.3334,1.5521,0.7842,-244.3326
7,2010-12-23 01:00,1.7180,1.6021,3.7352,4.6634,0.4692,0.6849,-2.3158
8,2010-12-24 01:00,4.3785,3.2846,9.5016,9.5489,0.7796,1.2809,-157.1342
9,2010-12-25 01:00,4.7090,3.5773,10.2497,10.4315,0.6494,1.0000,-54.6716


In [6]:
data1 = exp1.plot_model(arima1, plot="insample", return_data=True)
data2 = exp2.plot_model(arima2, plot="insample", return_data=True)
data3 = exp3.plot_model(arima3, plot="insample", return_data=True)